In [1]:
import pandas as pd
import numpy as np
import ast
number_list=[1,3,4,5,6]
for file_number in number_list:
    # 1. 데이터 읽기 (csv 파일 경로로 바꿔주세요)
    df_pred = pd.read_csv('./data/result/'+str(file_number)+'호선_혼잡도_전체예측결과.csv',encoding='euc-kr')      # 예측데이터
    df_real = pd.read_csv('C:/Users/user/Desktop/project/metrosafe/model/train/output날짜합치기/날짜합친'+str(file_number)+'호선.csv',encoding='euc-kr')     # 실데이터
    # 2. 키 기준 Inner join (일치하는 행만 남기기)
    on_cols = ['호선','역명','요일','시간','분','상하행']
    df_merge = pd.merge(
        df_pred, df_real,
        on=on_cols,
        suffixes=('_예측','_실제')
    )


    # 3. 혼잡도 리스트 파싱 및 로스율 계산
    def relative_loss(row):
        pred = np.array(ast.literal_eval(row['congestionCar_예측']))
        real = np.array(ast.literal_eval(row['congestionCar_실제']))

        # 분모 0 방지
        real_mod = np.where(real == 0, 1, real)
        loss = np.abs(pred - real) / real_mod * 100
        mean_loss = loss.mean()
        return loss.tolist(), mean_loss
    print(file_number)
    df_merge['로스율(상대)_10칸'], df_merge['로스율(상대)_평균'] = zip(*df_merge.apply(relative_loss, axis=1))

    # 4. 보기좋게 필요한 컬럼만 추출
    result = df_merge[
        on_cols + 
        ['congestionCar_예측','congestionCar_실제','로스율(상대)_10칸','로스율(상대)_평균']
    ]

    # 5. csv로 저장
    result.to_csv('./loss/'+str(file_number)+'호선_loss_result.csv', index=False,encoding='euc-kr')


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/project/metrosafe/model/train/output날짜합치기/날짜합친1호선.csv'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

# 1. CSV 불러오기 (또는 DataFrame 생성)
# 예: df = pd.read_csv('your_file.csv')
# 샘플 데이터가 이미 DataFrame이라면 바로 df 사용

# '로스율(상대)_평균'이 float이 아니라면 변환 필요:
# df['로스율(상대)_평균'] = df['로스율(상대)_평균'].astype(float)
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕' 폰트 지정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 부호(-) 정상 출력


for file_number in number_list:
    df = pd.read_csv('./loss/'+str(file_number)+'호선_loss_result.csv',encoding='euc-kr')
    # 2. 역별 평균 로스율 계산 (필요시 호선, 요일, 상하행 등 추가 그룹화 가능)
    station_loss = df.groupby('역명')['로스율(상대)_평균'].mean().sort_values()

    # 3. 바 차트 시각화
    plt.figure(figsize=(10,6))
    bars = plt.bar(station_loss.index, station_loss.values, color='skyblue')
    plt.title('역별 평균 상대 로스율(%)')
    plt.xlabel('역명')
    plt.ylabel('평균 상대 로스율(%)')
    plt.ylim(0, max(50, station_loss.max() + 5))

    # 4. 바 위에 숫자 표시
    for bar in bars:
        plt.text(
            bar.get_x() + bar.get_width() / 2, 
            bar.get_height(), 
            f'{bar.get_height():.1f}%', 
            ha='center', va='bottom'
        )
    plt.savefig('./loss/'+str(file_number)+'호선_loss_rate_chart.png', dpi=300)  # 🔸 이미지 저장
    plt.close()  # 🔸 그래프 초기화 (메모리 해제)


In [3]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

# 1. 사용할 호선 리스트
number_list = [1, 3, 4, 5, 6]

plt.rcParams["font.family"] = "Malgun Gothic"  # '맑은 고딕' 폰트 지정
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 부호(-) 정상 출력

for file_number in number_list:
    # == [A] 로스율 계산하기 ==
    # (1) 예측/실제 혼잡도 데이터 불러오기
    df_pred = pd.read_csv(
        f"./data/result/{file_number}호선_혼잡도_전체예측결과.csv", encoding="euc-kr"
    )
    df_real = pd.read_csv(
        f"C:/Users/user/metrosafe/model/train/output날짜합치기/날짜합친{file_number}호선.csv",
        encoding="euc-kr",
    )

    # (2) Inner Join
    on_cols = ["호선", "역명", "요일", "시간", "분", "상하행"]
    df_merge = pd.merge(df_pred, df_real, on=on_cols, suffixes=("_예측", "_실제"))

    # (3) 혼잡도 리스트 파싱 및 로스율(상대)_평균 계산
    def relative_loss(row):
        pred = np.array(ast.literal_eval(row["congestionCar_예측"]))
        real = np.array(ast.literal_eval(row["congestionCar_실제"]))
        real_mod = np.where(real == 0, 1, real)
        loss = np.abs(pred - real) / real_mod * 100
        mean_loss = loss.mean()
        return loss.tolist(), mean_loss

    print(file_number)
    df_merge["로스율(상대)_10칸"], df_merge["로스율(상대)_평균"] = zip(
        *df_merge.apply(relative_loss, axis=1)
    )

    # (4) 필요한 컬럼만 추출 및 저장
    result = df_merge[
        on_cols
        + [
            "congestionCar_예측",
            "congestionCar_실제",
            "로스율(상대)_10칸",
            "로스율(상대)_평균",
        ]
    ]
    # 로스율 컬럼 형 변환 (혹시 모를 타입 이슈 방지)
    result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)
    # (5) 저장
    result.to_csv(
        f"./loss/{file_number}호선_loss_result.csv", index=False, encoding="euc-kr"
    )

    # == [B] 역별 평균 로스율 시각화 ==
    # (6) 저장된 파일 불러오기
    df = pd.read_csv(f"./loss/{file_number}호선_loss_result.csv", encoding="euc-kr")

    # (7) 300% 이상 제외하는 필터 적용
    filtered_df = df[df["로스율(상대)_평균"] < 300]

    # (8) 역별 평균 로스율 계산
    station_loss = filtered_df.groupby("역명")["로스율(상대)_평균"].mean().sort_values()

    # (9) 바 차트 시각화 및 저장
    plt.figure(figsize=(10, 6))
    bars = plt.bar(station_loss.index, station_loss.values, color="skyblue")
    plt.title("역별 평균 상대 로스율(%)")
    plt.xlabel("역명")
    plt.ylabel("평균 상대 로스율(%)")
    plt.ylim(0, max(50, station_loss.max() + 5))
    plt.xticks(rotation=45, ha="right")

    # (10) 바 위에 숫자 표시
    for bar in bars:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            f"{bar.get_height():.1f}%",
            ha="center",
            va="bottom",
        )

    plt.tight_layout()
    plt.savefig(f"./loss/{file_number}호선_loss_rate_chart.png", dpi=300)
    plt.close()

1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)


3


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)


4


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:18: DtypeWarning: Columns (2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_real = pd.read_csv(


5


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)


6


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9404\3943405757.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["로스율(상대)_평균"] = result["로스율(상대)_평균"].astype(float)
